In [5]:
import pymongo
import base64
import numpy as np
from PIL import Image
from io import BytesIO

# Verbindung zur MongoDB herstellen
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Daten abrufen
data = collection.find()

# Canvas-Bilder dekodieren und in numpy Arrays umwandeln
def decode_canvas(canvas_data):
    image_data = base64.b64decode(canvas_data.split(",")[1])
    image = Image.open(BytesIO(image_data))
    return np.array(image)

# Zielgröße für alle Bilder festlegen (z.B. 35x280)
target_size = (35, 280)

# Funktion zum Resizing der Bilder
def resize_image(image, target_size):
    return np.array(Image.fromarray(image).resize(target_size, Image.BICUBIC))

# Alle Canvas-Bilder dekodieren und auf die Zielgröße bringen
canvases = []
labels = []
for record in data:
    for canvas in record["canvases"]:
        decoded_image = decode_canvas(canvas)
        resized_image = resize_image(decoded_image, target_size)
        canvases.append(resized_image)
        labels.append(record["username"])

canvases = np.array(canvases)
labels = np.array(labels)


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Bilder skalieren
canvases = canvases / 255.0

# Labels in numerische Werte umwandeln
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Daten in Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(canvases, labels, test_size=0.2, random_state=42)


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Modell erstellen
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(35, 280, 4)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Modell kompilieren
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modell zusammenfassen
model.summary()


C:\Users\Lion Witte\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 33, 278, 32)    │         1,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 139, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 137, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 68, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30464)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,899,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,919,587 (14.95 MB)

 Trainable params: 3,919,587 (14.95 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Modell trainieren
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 462ms/step - accuracy: 0.2778 - loss: 1.0337 - val_accuracy: 0.9167 - val_loss: 0.5069
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.7118 - loss: 0.5587 - val_accuracy: 0.8333 - val_loss: 0.3738
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.6181 - loss: 0.8285 - val_accuracy: 0.5000 - val_loss: 0.5825
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.7222 - loss: 0.4190 - val_accuracy: 0.5000 - val_loss: 0.4835
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.8403 - loss: 0.3701 - val_accuracy: 0.9167 - val_loss: 0.3197
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.7500 - loss: 0.3999 - val_accuracy: 0.9167 - val_loss: 0.3379
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.8021 - loss: 0.3588 - val_accuracy: 0.7500 - val_loss: 0.4484
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.8889 - loss: 0.3436 - val_accuracy: 0.8333 - val_loss:

In [10]:
# Modell evaluieren
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8333 - loss: 0.3607
Test accuracy: 0.8333333134651184


In [11]:
# Modell speichern
model.save("canvas_auth_model.h5")
